In [1]:
import ee
import ee.mapclient
from IPython.display import Image,display

In [2]:
# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

In [3]:
# Print the information for an image asset.
image = ee.Image('srtm90_v4')
print(image.getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0.0, -180.0, 0.0, -0.000833333333333, 60.0]}], 'version': 1494271934303000, 'id': 'srtm90_v4', 'properties': {'system:time_start': 950227200000, 'system:time_end': 951177600000, 'system:asset_size': 18827626666}}


In [4]:
url = image.getThumbURL({'min':0,'max':3000})
Image(url=url)

In [22]:
skysat = ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB')
print(skysat.getInfo())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [39]:
count = skysat.size()
print(count)

ee.Number({
  "type": "Invocation",
  "arguments": {
    "collection": {
      "type": "Invocation",
      "arguments": {
        "id": "SKYSAT/GEN-A/PUBLIC/ORTHO/RGB"
      },
      "functionName": "ImageCollection.load"
    }
  },
  "functionName": "Collection.size"
})


In [41]:
date_range = skysat.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', ee.Date(date_range.get('min')), ee.Date(date_range.get('max')))

Date range:  ee.Date({
  "type": "Invocation",
  "arguments": {
    "value": {
      "type": "Invocation",
      "arguments": {
        "dictionary": {
          "type": "Invocation",
          "arguments": {
            "collection": {
              "type": "Invocation",
              "arguments": {
                "id": "SKYSAT/GEN-A/PUBLIC/ORTHO/RGB"
              },
              "functionName": "ImageCollection.load"
            },
            "reducer": {
              "type": "Invocation",
              "arguments": {},
              "functionName": "Reducer.minMax"
            },
            "selectors": [
              "system:time_start"
            ]
          },
          "functionName": "Collection.reduceColumns"
        },
        "key": "min"
      },
      "functionName": "Dictionary.get"
    }
  },
  "functionName": "Date"
}) ee.Date({
  "type": "Invocation",
  "arguments": {
    "value": {
      "type": "Invocation",
      "arguments": {
        "dictionary": {
      

In [42]:
collection = ee.ImageCollection('LANDSAT/LT05/C01/T2')
  .filterDate('1987-01-01', '1990-05-01')
  .filterBounds(ee.Geometry.Point(25.8544, -18.08874));

IndentationError: unexpected indent (<ipython-input-42-b9b413af4e9d>, line 2)

In [26]:
point = ee.Geometry.Point(-70.905, 41.629) #-70.90519104980672,41.629932792735]

In [29]:
point_bounds = point.buffer(500000).bounds().getInfo()['coordinates']

In [30]:
point_bounds

[[[-76.90748735297683, 37.13118644518354],
  [-64.88317775799395, 37.13118644518354],
  [-64.88317775799395, 46.12843297574075],
  [-76.90748735297683, 46.12843297574075],
  [-76.90748735297683, 37.13118644518354]]]

In [32]:
# Reduce image collection to mean
skysat_min = skysat.mean()

In [36]:
# Calculate and plot NDVI 
# Note: Not using NDVI band, NDVI will be calculated from NIR and RED band
ndvi = skysat_min.normalizedDifference(["G", "R"])
Image(url=ndvi.getThumbUrl({
    'region':point_bounds,
    'min':0,
    'max':2550
}))